In [1]:
pip install streamlit sentence-transformers pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1

In [3]:
import streamlit as st
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util

# Load the SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Upload files in Streamlit
st.title("🔍 AI Resume Matcher")
st.write("Upload Resume and Job Description CSV files to match candidates with job roles.")

resume_file = st.file_uploader("Upload Resume CSV", type=["csv"])
jd_file = st.file_uploader("Upload Job Description CSV", type=["csv"])

if resume_file and jd_file:
    # Load CSV files
    df_resume = pd.read_csv(resume_file)
    df_jd = pd.read_csv(jd_file)

    # Preprocess column names
    df_resume.columns = df_resume.columns.str.strip()
    df_jd.columns = df_jd.columns.str.strip()

    # Ensure necessary columns exist
    if "Skills" not in df_resume.columns or "Keywords" not in df_jd.columns:
        st.error("CSV files must contain 'Skills' in Resume and 'Keywords' in JD.")
    else:
        # Fill missing values
        df_resume["Skills"] = df_resume["Skills"].fillna("").astype(str)
        df_jd["Keywords"] = df_jd["Keywords"].fillna("").astype(str)

        # Encode using SBERT
        resume_embeddings = model.encode(df_resume["Skills"].tolist(), convert_to_tensor=True)
        jd_embeddings = model.encode(df_jd["Keywords"].tolist(), convert_to_tensor=True)

        # Compute cosine similarity
        similarity_scores = util.cos_sim(resume_embeddings, jd_embeddings)

        # Convert similarity matrix to a DataFrame
        similarity_df = pd.DataFrame(similarity_scores.cpu().numpy(),
                                     index=df_resume["Resume ID"],
                                     columns=df_jd["Job Role"])

        # Find best matches
        top_matches = similarity_df.idxmax(axis=1)
        top_scores = similarity_df.max(axis=1)

        # Create results DataFrame
        results_df = pd.DataFrame({
            "Resume ID": df_resume["Resume ID"],
            "Best Matching Job Role": top_matches.values,
            "Similarity Score": top_scores.values
        })

        # Display results in Streamlit
        st.subheader("🔹 Matching Results")
        st.write(results_df)

        # Download results as CSV
        st.download_button(label="📥 Download Results", data=results_df.to_csv(index=False),
                           file_name="Resume_Job_Matching.csv", mime="text/csv")


2025-03-06 13:34:35.917 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 13:34:35.921 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 13:34:35.926 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 13:34:35.928 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 13:34:35.929 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 13:34:35.931 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 13:34:35.932 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 13:34:35.933 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [4]:
!streamlit run app.py & npx localtunnel --port 8501

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py
⠙⠹⠸⠼⠴⠦⠧⠇Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) ^C
